## Align Zoom and Notebook Side By Side

![split_screenshot.png](img/split_screenshot.png)

<img src="img/heading_tutorial.jpg" style="width: 100%; margin:0;" />

<img src="img/tab_intro.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
! echo "Hello from the Linux Shell"

! echo "\n✅ Step Complete\n"

<br /><br />

<br /><br />

<img src="img/heading_overview.jpg" style="width: 100%; margin:0;" />

<br /><br />

<img src="img/heading_domain.jpg" style="width: 100%; margin:0;" />

<br /><br />

<img src="img/tab_install_pip.png" alt="tab" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
! sudo apt update && sudo apt install python3-pip

! echo "\n✅ Step Complete\n"

<br /><br />

<img src="img/tab_install_hagrid.png" alt="tab" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
! pip install -U hagrid

! echo "\n✅ Step Complete\n"

<br /><br />

<img src="img/tab_install_syft.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
! pip install --pre -U syft

! echo "\n✅ Step Complete\n"

<br /><br />

<img src="img/tab_launch_domain.png" alt="tab" style="width: 664px; margin:0;" />

In [ ]:
# edit DOMAIN_NAME and run this cell

DOMAIN_NAME = "Canada"

! hagrid launch {DOMAIN_NAME} to docker:80 --tag=latest --tail=false --silent

! echo "\n✅ Step Complete\n"

<br /><br />

<img src="img/tab_check_domain.png" alt="tab" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
! hagrid check "localhost:80" --wait --silent

! echo "\n✅ Step Complete\n"

<br /><br />

<br /><br />

<img src="img/heading_data.jpg" style="width: 100%; margin:0;" />

<br /><br />

<img src="img/tab_import_syft.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
import syft as sy
from utils import *
print("Syft is imported")

<br /><br />

<img src="img/tab_python_client_login.png" style="width: 664px; margin:0;" />

In [ ]:
domain_client = sy.login(
    url="localhost:80",#auto_detect_domain_host_ip(),
    email="info@openmined.org",
    password="changethis"
)

<br /><br />

<img src="img/tab_get_dataset.png" style="width: 664px; margin:0;" />

In [ ]:
# add url to MY_DATASET_URL then run this cell

MY_DATASET_URL = "https://raw.githubusercontent.com/OpenMined/PySyft/dev/notebooks/trade_demo/datasets/ca%20-%20feb%202021.csv"

trade_dataset = download_dataset_as_dataframe(MY_DATASET_URL)

<br /><br />

<img src="img/tab_preview_dataset.png" style="width: 664px; margin:0;" />

In [ ]:
trade_dataset.head()

<br /><br />

<img src="img/tab_preprocess_data.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell

# Filter trade details of imported commodities
imports = trade_dataset[trade_dataset['Trade Flow'] == "Imports"]

# Filter trade details of exported commodities
exports = trade_dataset[trade_dataset['Trade Flow'] == "Exports"]

In [ ]:
# run this cell

# Extract the trade value of the imported commodities
trade_value_of_imported_commodites = imports['Trade Value (US$)'].to_numpy()

# Extract the trade value of the exported commodities
trade_value_of_exported_commodites = exports['Trade Value (US$)'].to_numpy()

<br /><br />

<img src="img/tab_annotate_import.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell

# Filter out the countries from which commodities were imported
countries_from_which_commodities_are_imported = imports["Partner"]

data_subjects_for_imported_commodities = sy.DataSubjectArray.from_objs(
    countries_from_which_commodities_are_imported
)

# annotate traded value of imported commodities with dp metadata
imports_data = sy.Tensor(
    trade_value_of_imported_commodites
).annotated_with_dp_metadata(min_val=0, max_val=255, data_subjects=data_subjects_for_imported_commodities)

<br /><br />

<img src="img/tab_annotate_export.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell

# Filter out the countries to which commodities were exported
countries_to_which_commodities_are_exported = exports["Partner"]

data_subjects_for_exported_commodities = sy.DataSubjectArray.from_objs(countries_to_which_commodities_are_exported)


# annotate traded value of exported commodities with dp metadata
exports_data = sy.Tensor(
    trade_value_of_exported_commodites
).annotated_with_dp_metadata(min_val=0, max_val=1, data_subjects=data_subjects_for_exported_commodities)

<br /><br />

<img src="img/tab_upload_dataset.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell

# Upload data to the domain
domain_client.load_dataset(
    name="Imports and Exports Trade dataset Feb 2021",
    assets={
        "imports": imports_data, 
        "exports": exports_data
    },
    description="This dataset contains information about all of imports and exports \
    in the month of Feb 2021. Measured in USD.",
)

<br /><br />

<img src="img/tab_check_dataset.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
domain_client.datasets

<br /><br />

<br /><br />

<img src="img/heading_network.jpg" style="width: 100%; margin:0;" />

<br /><br />

<img src="img/tab_browse_networks.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
sy.networks

<br /><br />

<img src="img/tab_join_network.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
NETWORK_NAME = "UN PET Lab Network"
network_client = sy.networks[NETWORK_NAME]
domain_client.apply_to_network(network_client)

<br /><br />

<img src="img/tab_see_domains.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
network_client.domains

<br /><br />

<br /><br />

<img src="img/heading_account.jpg" style="width: 100%; margin:0;" />

<br /><br />

<img src="img/tab_create_user.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell
data_scientist_details = domain_client.create_user(
    name="Sam Carter",
    email="sam@stargate.net",
    password="changethis",
    budget=99999
)

<br /><br />

<img src="img/tab_copy_details.png" style="width: 664px; margin:0;" />

In [ ]:
# run this cell then copy the output
print("Please give these details to the Data Scientist 👇🏽")
print(data_scientist_details)

<br /><br />

<img src="img/heading_recap.jpg" style="width: 100%; margin:0;" />

<br /><br />

# Thank You

If you have any questions for our team please don't hesitate to reach out via email or the slack link below.

# Links

🌍 Web:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; https://blog.openmined.org/  
💬 Slack:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; https://openmined.slack.com/  
🎥 Course:&nbsp;&nbsp; https://courses.openmined.org/  
📰 Blog:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; https://blog.openmined.org/  
🐙 Code:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; https://github.com/OpenMined/PySyft